## 2. High-level Pseudocode

In [ ]:
def getCentroid(points):
    pass

def convexHullEdge(...):




    list<pair<int, int>> convexHull(list<pair<int, int>> points) {
        
        // Find a centroid point amongst the given points
        centroid = getCentroid(points)

        // Define a reference point relative to the centroid
        ref = getReference(centroid)

        //
        pairs = []
        for point in points {
            // First, define which region the point falls
            left = point.x <= centroid.x

            // 
            intAngle = getInteriorAngle(centroid, point, ref)
            if (!left) {
                intAngle = 360 - intAngle
            }
            Pair<Point, Angle> pt_ang_pair = Pair(point, intAngle)
            pairs.append(pt_ang_pair)
        }

        // Sort the pairs
        sorted_pairs = sort(pairs)
        sorted_ll = DoublyLinkedList(sorted_pairs)

        // For each pair, form a quadrilater, and determine if the interior angle is convex

        return_points = []
        i = 1
        prevNode = sorted_ll.head()
        currNode = prevNode.next()
        nextNode = currNode.next()
        while nextNode.hasNext() {
            intAngle = getInteriorAngleQuad(prevNode.data(), currNode.data(), nextNode.data(), centroid)

            //
            if intAngle < 180 {
                return_points.append(currNode.data)
            } else {
                newIntAngle = intAngle
                while(prevNode.hasPrev && newIntAngle >= 180) {
                    prevNode.setNext(nextNode)
                    nextNode.setPrev(prevNode)

                    currNode = prevNode
                    prevNode = currNode.prev()

                    newIntAngle = getInteriorAngleQuad(prevNode, currNode, nextNode, centroid)
                } 
            }

            nextNode = nextNode.next()
            currNode = currNode.next()
            prevNode = prevNode.next()
        }

        // Make adjustments for the final point
        currNode = currNode.next()
        nextNode = sorted_ll.first()
        prevNode = prevNode.next()

        intAngle = getInteriorAngleQuad(prevNode, currNode, nextNode, centroid)
        
        if intAngle < 180 {
            return_points.append(currNode.data)
        } else {
            newIntAngle = intAngle
            while(prevNode.hasPrev && newIntAngle >= 180) {
                prevNode.setNext(nextNode)
                nextNode.setPrev(prevNode)

                currNode = prevNode
                prevNode = currNode.prev()

                newIntAngle = getInteriorAngleQuad(prevNode, currNode, nextNode, centroid)
            } 
        }

        return return_points
    }

